In [ ]:
!pip install -U bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)



Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Loading model...


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# =========================
# BƯỚC 1: SINH THINK + FINAL
# =========================

messages = [
    {
        "role": "system",
        "content": (
            "Bạn là một trợ lý AI chuyên suy luận logic.\n\n"
            "Khi trả lời, hãy làm theo đúng định dạng sau:\n\n"
            "<think>\n"
            "Trình bày chi tiết toàn bộ quá trình suy luận từng bước.\n"
            "</think>\n\n"
            "<final>\n"
            "Đưa ra câu trả lời cuối cùng ngắn gọn.\n"
            "</final>\n\n"
            "Luôn trả lời bằng tiếng Việt."
        )
    },
    {
        "role": "user",
        "content": "Một người nông dân có gà và bò. Tổng cộng có 20 con vật và 56 cái chân. Hỏi có bao nhiêu con gà và bao nhiêu con bò?"
    }
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9
    )

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Cắt phần trước <｜Assistant｜>
if "<｜Assistant｜>" in decoded:
    decoded = decoded.split("<｜Assistant｜>")[-1]

# =========================
# PARSE THINK + FINAL
# =========================

thinking = ""
final_answer = ""

if "<think>" in decoded and "</think>" in decoded:
    thinking = decoded.split("<think>")[1].split("</think>")[0].strip()

if "<final>" in decoded and "</final>" in decoded:
    final_answer = decoded.split("<final>")[1].split("</final>")[0].strip()
else:
    final_answer = decoded.strip()

print("\n" + "-"*50)
print("🧠 SUY LUẬN (THINK):")
print(thinking)

print("\n" + "-"*50)
print("✅ KẾT QUẢ CUỐI (FINAL):")
print(final_answer)

# =========================
# BƯỚC 2: SINH REFLECTION (SELF-CHECK)
# =========================

reflection_messages = [
    {
        "role": "system",
        "content": (
            "Bạn là một trợ lý AI chuyên kiểm tra lời giải toán.\n\n"
            "Nhiệm vụ của bạn:\n"
            "- Đọc kỹ lời giải bên dưới\n"
            "- Kiểm tra từng bước suy luận\n"
            "- Xác nhận kết quả đúng hay sai\n"
            "- Nếu có lỗi, chỉ ra lỗi và sửa lại\n\n"
            "Chỉ trả lời trong thẻ:\n\n"
            "<reflection>\n"
            "...\n"
            "</reflection>\n\n"
            "Luôn trả lời bằng tiếng Việt."
        )
    },
    {
        "role": "user",
        "content": (
            "Đây là lời giải của mô hình:\n\n"
            f"{thinking}\n\n"
            f"Kết quả cuối cùng:\n{final_answer}\n\n"
            "Hãy kiểm tra lại lời giải này."
        )
    }
]

text_reflect = tokenizer.apply_chat_template(
    reflection_messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs_reflect = tokenizer(text_reflect, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs_reflect = model.generate(
        **inputs_reflect,
        max_new_tokens=256*10,
        temperature=0.5,
        top_p=0.9
    )

decoded_reflect = tokenizer.decode(outputs_reflect[0], skip_special_tokens=True)

# Cắt phần trước <｜Assistant｜>
if "<｜Assistant｜>" in decoded_reflect:
    decoded_reflect = decoded_reflect.split("<｜Assistant｜>")[-1]

reflection = ""

if "<reflection>" in decoded_reflect and "</reflection>" in decoded_reflect:
    reflection = decoded_reflect.split("<reflection>")[1].split("</reflection>")[0].strip()
else:
    reflection = decoded_reflect.strip()

print("\n" + "-"*50)
print("🔍 PHẢN TƯ (REFLECTION - SELF CHECK):")
print(reflection)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



--------------------------------------------------
🧠 SUY LUẬN (THINK):
B scaffold
Để giải bài toán này, ta sẽ sử dụng phương pháp hệ số và phương trình linear.

Đầu tiên, ta defi nition các biến:
- Gọi số gà là \( x \).
- Gọi số bò là \( y \).

Từ đề bài, ta có hai điều kiện:
1. Tổng số con vật là 20:
   \[
   x + y = 20
   \]
2. Tổng số chân là 56:
   \[
   2x + 4y = 56
   \]

Tiếp theo, ta sẽ giải hệ phương trình này để tìm giá trị của \( x \) và \( y \).

Từ phương trình đầu tiên:
\[
x = 20 - y
\]

Thay giá trị này vào phương trình thứ hai:
\[
2(20 - y) + 4y = 56
\]
\[
40 - 2y + 4y = 56
\]
\[
2y = 16
\]
\[
y = 8
\]

Sau đó, thay \( y = 8 \) vào phương trình đầu tiên:
\[
x + 8 = 20
\]
\[
x = 12
\]

Vậy, nông dân có 12 con gà và 8 con bò.

--------------------------------------------------
✅ KẾT QUẢ CUỐI (FINAL):
<think>
B scaffold
Để giải bài toán này, ta sẽ sử dụng phương pháp hệ số và phương trình linear.

Đầu tiên, ta defi nition các biến:
- Gọi số gà là \( x \).
- Gọi số bò là \

In [ ]:
from transformers import TextIteratorStreamer
import threading

streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)

generation_kwargs = dict(
    **inputs,
    max_new_tokens=256*5,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer
)

thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

print("\n🧠 MODEL IS THINKING...\n")

buffer = ""
in_think = False

final_answer = ""

for new_text in streamer:
    buffer += new_text

    # Khi bắt đầu <think>
    if "<think>" in buffer:
        in_think = True
        buffer = buffer.split("<think>")[-1]
        continue

    # Khi kết thúc </think>
    if "</think>" in buffer:
        in_think = False
        buffer = buffer.split("</think>")[-1]
        print("\n\n✅ FINAL ANSWER:\n")
        continue

    if in_think:
        # Hiện thinking live
        print(new_text, end="", flush=True)
    else:
        # Đây là phần final answer
        final_answer += new_text
        print(new_text, end="", flush=True)

print("\n\n🎯 DONE.")
